In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:80% !important;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
</style>
<style>
span.toc-item-num{
display:none;
}
</style>
"""))

<b><font color="red" size="6">ch14. 웹데이터수집</font></b>
# 1절. BeautifulSoup과 parser

`pip install bs4` 아나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함



In [2]:
import requests
from requests_file import FileAdapter

In [3]:
s= requests.Session()# HTTP 요청관리를 위한 세션객체
s.mount("file://",FileAdapter())
response= s.get('file:///ai/source/07_python/ch14_sample.html')
response

<Response [200]>

In [4]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta charset="UTF-8">\r\n<body>\r\n    <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n    <h1 class="css">Hi, CSS</h1>\r\n    <div id="subject">subject 선택자 안의 내용</div>\r\n    <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n    <div class="contents">\r\n        선택자를 어떻게 작성하는냐에 따라\r\n        <span>다른<b>요소가 반환</b></span> 됩니다\r\n    </div>\r\n    <div>CSS 선택자는 다양한 곳에<b>활용</b>됩니다</div>\r\n</body>\r\n</html>\r\n'

In [5]:
response.text

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta charset="UTF-8">\r\n<body>\r\n    <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n    <h1 class="css">Hi, CSS</h1>\r\n    <div id="subject">subject 선택자 안의 내용</div>\r\n    <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n    <div class="contents">\r\n        선택자를 어떻게 작성하는냐에 따라\r\n        <span>다른<b>요소가 반환</b></span> 됩니다\r\n    </div>\r\n    <div>CSS 선택자는 다양한 곳에<b>활용</b>됩니다</div>\r\n</body>\r\n</html>\r\n'

In [6]:
# html파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, # response.text
                    "html.parser")
# soup


In [7]:
# soup.select_one('선택자') : 해당 선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el:',el)
print('el.text:',el.text)
print('el.string:',el.string)
print('el의 속성들:',el.attrs)
print('el의 class속성:',el.attrs['class']) # class 속성은 하나라도 list로 생성
print('el의 name:',el.name)

el: <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text: Hello, CSS
el.string: Hello, CSS
el의 속성들: {'class': ['greeting', 'css'], 'id': 'text'}
el의 class속성: ['greeting', 'css']
el의 name: h1


In [8]:
# soup.select('선택자') : 해당 선택자 모든 엘리먼트를 리스트로
el = soup.select('h1')
print('el:',el)
print('el들의 text:',[e for e in el])
print('el들의 string :', [e.string for e in el])
print('el들의 속성들 :', [e.attrs for e in el])
print('el들의 class 속성 :', [e.attrs['class'] for e in el])

el: [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
el들의 text: [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
el들의 string : ['Hello, CSS', 'Hi, CSS']
el들의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]
el들의 class 속성 : [['greeting', 'css'], ['css']]


In [9]:
#select('선택자')와 find_all(태그)
print('모든 h1, span태그(select):',soup.select('h1,span'),sep='\n')
print('모든 h1, span태그(find_all):',soup.find_all(['h1','span'],
                                              class_=['css']),sep='\n')
#select_all('선택자')와 find(태그)
print('select_one:', soup.select_one('h1.css'),sep='\n')
print('find:', soup.find('h1',class_='css'),sep='\n')

print('find:', soup.find_all('h1',class_='css'),sep='\n')
print('find:', soup.find('a',class_='css'),sep='\n')

모든 h1, span태그(select):
[<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
모든 h1, span태그(find_all):
[<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
select_one:
<h1 class="greeting css" id="text">Hello, CSS</h1>
find:
<h1 class="greeting css" id="text">Hello, CSS</h1>
find:
[<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find:
None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 request모듈(GET 요청) / urlib.request 모듈(urlopen)

In [10]:
# 크롤링 허용범위마다 사이트마다 ~/robots.txt에서 확인할 수 있습니다
    # User-agent: 모든 크롤링을 허용(*), 지침을 적용할 클로러를 지정
    # Allow,Disallow: 크롤링을 제한할 경로

In [11]:
# 방법1
import requests
response = requests.get('http://api.github.com')
response, response.status_code

(<Response [200]>, 200)

In [12]:
jsondata = response.json()
type(jsondata)

dict

In [13]:
# 방법2
from urllib.request import urlopen
response = urlopen('http://api.github.com')
response.status

200

# 2.2  BeautifulSoup 모듈을 활용 html 웹 데이터 수집
### 1) 환율정보 가져오기(네이버금융>시장지표)
 * https://finance.naver.com/marketindex/

In [14]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response=requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')
price= soup.select('div.head_info> span.value')
[round(float(''.join(p.text.split(','))),2) for p in price]
[round(float(p.text.replace(',','')),2)for p in price]

[1427.8,
 945.09,
 1507.4,
 196.91,
 150.92,
 1.06,
 1.28,
 106.12,
 68.37,
 1645.82,
 2685.8,
 122558.1]

In [15]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response=urlopen(url)
soup = BeautifulSoup(response,'html.parser')


In [16]:
price= soup.select('div.head_info> span.value')
[round(float(''.join(p.text.split(','))),2) for p in price]
[round(float(p.text.replace(',','')),2)for p in price]

[1427.8,
 945.09,
 1507.4,
 196.91,
 150.92,
 1.06,
 1.28,
 106.12,
 68.37,
 1645.82,
 2685.8,
 122558.1]

In [17]:
title = soup.select('h3.h_lst > span.blind')
for t in title:
    print(t.text,end='\t')

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [18]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7,'')
unit

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [19]:
status = soup.select('div.head_info> span.blind')
for idx in range(len(status)):
    print(status[idx].text,end='\t')

하락	하락	하락	하락	상승	상승	상승	상승	상승	상승	상승	하락	

In [20]:
len(title),len(price),len(unit),len(status)

(12, 12, 12, 12)

In [21]:
for idx in range(len(title)):
    print("{}:{} {} - {}".format(title[idx].text,price[idx].text,unit[idx],status[idx].text))

미국 USD:1,427.80 원 - 하락
일본 JPY(100엔):945.09 원 - 하락
유럽연합 EUR:1,507.40 원 - 하락
중국 CNY:196.91 원 - 하락
달러/일본 엔:150.9200 엔 - 상승
유로/달러:1.0580 달러 - 상승
영국 파운드/달러:1.2779 달러 - 상승
달러인덱스:106.1200  - 상승
WTI:68.37 달러 - 상승
휘발유:1645.82 원 - 상승
국제 금:2685.8 달러 - 상승
국내 금:122558.1 원 - 하락


In [22]:
for t,p,u,s in zip(title,price,unit,status):
    print("{}:{} {} - {}".format(t.text,p.text,u,s.text))

미국 USD:1,427.80 원 - 하락
일본 JPY(100엔):945.09 원 - 하락
유럽연합 EUR:1,507.40 원 - 하락
중국 CNY:196.91 원 - 하락
달러/일본 엔:150.9200 엔 - 상승
유로/달러:1.0580 달러 - 상승
영국 파운드/달러:1.2779 달러 - 상승
달러인덱스:106.1200  - 상승
WTI:68.37 달러 - 상승
휘발유:1645.82 원 - 상승
국제 금:2685.8 달러 - 상승
국내 금:122558.1 원 - 하락


### 2) 이번주 로또번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin
- 1148회 (2024년 11월 20일 추천)
당첨번호 : [3,6,13,15,16,22]
보너스번호 : 32

In [23]:
# 방법2
from urllib.request import urlopen
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response=urlopen(url)
soup = BeautifulSoup(response,'html.parser')


In [24]:
# times = soup.select_one('div.win_result strong').text # 1148회
winresult= soup.find('div',class_='win_result')
times = winresult.find('strong').text
date = winresult.find('p',class_='desc').text
print(times)
# date = soup.select_one('div.win_result p.desc').text # > 를 쓰고싶으면 바로 밑에 있는 자식이어야한다
print(times,date)
# lotto_number = soup.select('div.num.win span')
win = soup.find('div',class_=['win'])
lotto_numbers= win.find_all('span')
print('로또번호 :',[int(lotto.text) for lotto in lotto_numbers])

# bonus_number = soup.select_one('div.num.bonus span').text
bonus = soup.find('div',class_='bonus')
bonus_numbers = bonus.find('span').text
print('보너스 번호:',bonus_number)

1149회
1149회 (2024년 12월 07일 추첨)
로또번호 : [8, 15, 19, 21, 32, 36]


NameError: name 'bonus_number' is not defined

### 3) 다음 검색 리스트 가져오기
```
no title        href
- 비트코인 뉴스 - https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8
- "비트코인 8800만원에 주운 사람이 승자"…한밤 불태운 코인 투자자들 - https://v.daum.net/v/20241204113454203
- [속보] 비상계엄 선포에 비트코인 '쇼크'…30% 급락 https://v.daum.net/v/20241203231205730

```

In [ ]:
import requests
from bs4 import BeautifulSoup
url ='https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8'
response = requests.get(url)
soup = BeautifulSoup(response.text,"html.parser")
items_find_list=[] # 검색한 결과를 담을 dict 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append({'no':idx,
                           'title':item.text,
                           'link':item.attrs['href']})

items_find_list
import pandas as pd
pd.DataFrame(items_find_list)


In [ ]:
import requests
from bs4 import BeautifulSoup
url ='https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8'
response = requests.get(url)
soup = BeautifulSoup(response.text,"html.parser")
items_find_list=[] # 검색한 결과를 담을 dict 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx,item.text,item.attrs['href']])

#print(items_find_list)
import pandas as pd
pd.DataFrame(items_find_list,columns=['no','title','link'])


In [ ]:
items_find_list=[] # 검색한 결과를 담을 dict 리스트('div.item-title > strong.tit-g > a')
item_titles = soup.find_all('div',class_='item-title')

for idx, item in enumerate(item_titles):
    strong = item.find('strong')
    if strong:
        a = strong.find('a')
        items_find_list.append([idx, a.text,a.attrs['href']])
pd.DataFrame(items_find_list, columns=['no','title','link'])        



In [ ]:
# 다음 검색 (원하는 키워드를 원하는 페이지 수 만큼)
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
# print 하는 함수
def collect_list(keyword, page):
    '다음 검색을 keyword로 한 결과를 출력 '
#   url = "https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q={}&p={}".format(keyword,page)
#   response = requests.get(url)
    url = "https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_y"
    response = requests.get(url,params = {'q':keyword,'p':page})
    soup = BeautifulSoup(response.text,"html.parser")
    items_find_list=[] # 검색한 결과를 담을 dict 리스트('div.item-title > strong.tit-g > a')
    item_titles = soup.find_all('div',class_='item-title')

    for idx, item in enumerate(item_titles):
        strong = item.find('strong')
        if strong:
            a = strong.find('a')
            items_find_list.append([(page-1)*10+idx+1, a.text,a.attrs['href']])
            print((page-1)*10+idx, a.text,a['href'])



In [ ]:
collect_list('청바지',1)

In [ ]:
import time
keywords = ['청바지','닭가슴살']
pages = 3
for keyword in keywords:
    print(f" {keyword} 검색결과입니다")
    for page in range(1,pages+1):
        collect_list(keyword,page)
        time.sleep(1) # 1초 잠시 쉼

In [ ]:
# 다음 검색 (원하는 키워드를 원하는 페이지 수 만큼)
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
# print 하는 함수
def collect_list_return(keyword, page):
    '다음 검색을 keyword로 한 결과를 출력 '
#   url = "https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q={}&p={}".format(keyword,page)
#   response = requests.get(url)
    url = "https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_y"
    response = requests.get(url,params = {'q':keyword,'p':page})
    soup = BeautifulSoup(response.text,"html.parser")
    items_find_list=[] # 검색한 결과를 담을 dict 리스트('div.item-title > strong.tit-g > a')
    item_titles = soup.find_all('div',class_='item-title')

    for idx, item in enumerate(item_titles):
        strong = item.find('strong')
        if strong:
            a = strong.find('a')
            items_find_list.append([(page-1)*10+idx+1, a.text,a.attrs['href']])
#            items_find_list.append({
#                'no':(page-1)*10+idx,
#                'title':a.text,
#                'link':a['href']
#            })
            #print((page-1)*10+idx, a.text,a['href'])
            if idx == 9 :
                print(page,'페이지 완료')
    return items_find_list



In [ ]:
temp = collect_list_return('청바지',2)

In [ ]:
import pandas as pd
pd.DataFrame(collect_list_return('청바지',2),
             columns=['no','title','link'])

In [ ]:
keywords = ['청바지','닭가슴살']
result0=[]# 청바지 1~pages까지 검색결과를 담을 list
result1=[]# 닭가슴살 1~pages까지 검색결과를 담을 list

pages = 3
for i, keyword in enumerate(keywords):
    print(f'============{i+1}번째 검색어 {keyword}검색결과 수집중입니다')
    for page in range(1,pages+1):
        if i==0:
            result0.extend(collect_list_return(keyword,page))
        elif i==1:
            result1.extend(collect_list_return(keyword,page))

In [ ]:
result0_df=pd.DataFrame(result0)
result1_df=pd.DataFrame(result1)


In [ ]:
result0_df.to_csv('data/ch14_'+keywords[0]+'.csv',index=False)
result1_df.to_csv('data/ch14_'+keywords[1]+'.csv',index=False)

### 4) User-Agent를 추가하여 크롤링
- 크롤링을 막는 사이트
-https://www.melon.com/robots.txt에서 

In [ ]:
# 방법1 
from urllib.request import urlopen
from bs4 import BeautifulSoup
url = "https://www.melon.com/chart/"
#melonpage = urlopen(url) #error

In [ ]:
import requests
from bs4 import BeautifulSoup
url = "https://www.melon.com/chart/"
melonpage = requests.get(url)
melonpage
soup = BeautifulSoup(melonpage.text,"html.parser")
soup

In [ ]:
# User-Agent 를 추가
import requests
from bs4 import BeautifulSoup
url = "https://www.melon.com/chart/"
headers= {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36' }
melonpage= requests.get(url,headers = headers)
# melonpage
soup =BeautifulSoup(melonpage.text, # melon.context
                    "html.parser")
title_els = soup.select('div.ellipsis.rank01 > span')
divels = soup.find('div', class_='rank01')
singer_els = soup.select('div.ellipsis.rank02 > span.checkEllipsis')
#[s.text for s in singer_els]
[t.text.strip() for t in title_els]
#for title_el, singer_el in zip(title_els,singer_els):
#    print(title_el.text, '|', singer_el.text)
#for idx in range(len(title_els)):
#    print("{}위 | {}-{}".format(idx+1,title_els[idx].text.strip(),
#                             singer_els[idx].text))

for idx,(title_el, singer_el) in enumerate(zip(title_els,singer_els)):
    print("{}위 | {}-{}".format(idx+1,
                               title_el.text.strip(),
                             singer_el.text))

###  5)네이버 지식인 검색(open API 사용X)
- 특정키워드를 특정페이지 수(n페이지)만큼


In [ ]:
# 방법 1
from urllib.request import urlopen
from urllib.parse import quote
keyword=quote('쳇지피티')
# 쳇지피티 =>%EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0(url 인코딩 방식으로 전환)
url = f"https://kin.naver.com/search/list.naver?query={keyword}"
response = urlopen(url)
print(url)
print(response.status)


In [ ]:
# 방법 2
from requests import get
keyword='쳇지피티'
url = f"https://kin.naver.com/search/list.naver?query={keyword}"
response = get(url)
print(url)
print(response.status_code)

In [ ]:
# paging 처리 포함한 (select 사용)
keyword= '쳇지피티'
pages = 3
items_list = [] # 크롤링한 데이터를 담을 list
for page in range(1,pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    # 글 제목과 link를 가져와서 items_list
    response = requests.get(url)
    soup = BeautifulSoup(response.text,"html.parser")
    items_find_list=[] # 검색한 결과를 담을 dict 리스트
    items_el = soup.select('ul.basic1 >li dl dt > a')
    for idx, item in enumerate(items_el):
        items_find_list.append({'title':item.text,
                           'link':item.attrs['href']})

items_find_list
import pandas as pd
pd.DataFrame(items_find_list)



In [ ]:
# paging 처리 포함한 (find 사용)
keyword= '쳇지피티'
pages = 1
items_list = [] # 크롤링한 데이터를 담을 list
for page in range(1,pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    # 글 제목과 link를 가져와서 items_list
    response = requests.get(url)
    soup = BeautifulSoup(response.text,"html.parser")
    dt_els = soup.find_all('dt')
    for dt_el in dt_els:
        item=dt_el.find('a')
        items_list.append({'title': item.text,
                          'link': item.attrs['href']})
items_list
import pandas as pd
pd.DataFrame(items_list)



###  6)네이버 지식인 검색(open API 사용O)
- 특정키워드를 특정여러 데이터(n개)만큼


In [ ]:
import os
import sys
import urllib.request
client_id = "iCI8iNLX2KTckJkLS66U"
client_secret = "G67mYmxEIL"
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin.json?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/kin.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [ ]:
import requests
import pandas as pd
import json # response 텍스트를 json딕셔너리로
client_id = "iCI8iNLX2KTckJkLS66U"
client_secret = "G67mYmxEIL"
query = '쳇지피티'
items_list = []
url = f"https://openapi.naver.com/v1/search/kin.json?query={query}&display=30"
headers = {"X-Naver-Client-ID":client_id,
          "X-Naver-Client-Secret":client_secret}
response = requests.get(url,headers=headers)
#print(response.text[:500])
items = json.loads(response.text)['items']
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link = item['link']
    description = item['description'].replace('<b>','').replace('</b>','')
    items_list.append([title,link,description])
pd.DataFrame(items_list,columns=['title','link','description'])

### quiz) 네이버 open API를 이용해서 청바지 이미지 100건의 데이터를 img_list.csv 파일로

In [ ]:
def get_image_list(query):
    import requests
    import json # response 텍스트를 json딕셔너리로
    client_id = "iCI8iNLX2KTckJkLS66U"
    client_secret = "G67mYmxEIL"
    image_list = []
    url = f"https://openapi.naver.com/v1/search/image.json?query={query}&display=100"
    headers = {"X-Naver-Client-ID":client_id,
              "X-Naver-Client-Secret":client_secret}
    response = requests.get(url,headers=headers)
    #print(response.text[:500])
    items = json.loads(response.text)['items']
    for item in items:
        title = item['title']
        link = item['link']
        thumbnail = item['thumbnail']
        sizeheight = item['sizeheight']
        sizewidth = item['sizewidth']
        image_list.append([title,link,thumbnail,sizeheight,sizewidth])
        #link 파일을 n.jpg 저장
    return image_list

In [ ]:
img_list =get_image_list('닭가슴살')
df = pd.DataFrame(img_list,columns=['제목','링크','주소','넓이','높이'])
df

In [ ]:
def save_img(i,url):
    file_extension = url.split('.')[-1]
    quote_index =file_extension.find('?')
    if quote_index !=-1:
        file_extension = file_extension[:quote_index]
    img = requests.get(url).content
    open('image/'+str(i+1)+'.'+file_extension,'wb').write(img)

In [ ]:
save_img(0,df.loc[0,'링크'])

In [ ]:
save_img(9,df.loc[9,'링크'])

In [ ]:
def get_image_list_save_img(query):
    import requests
    import pandas as pd
    import json # response 텍스트를 json딕셔너리로
    client_id = "iCI8iNLX2KTckJkLS66U"
    client_secret = "G67mYmxEIL"
    image_list = []
    url = f"https://openapi.naver.com/v1/search/image.json?query={query}&display=100"
    headers = {"X-Naver-Client-ID":client_id,
              "X-Naver-Client-Secret":client_secret}
    response = requests.get(url,headers=headers)
    #print(response.text[:500])
    items = json.loads(response.text)['items']
    for idx, item in enumerate(items):
        title = item['title']
        link = item['link']
        thumbnail = item['thumbnail']
        sizeheight = item['sizeheight']
        sizewidth = item['sizewidth']
        image_list.append([title,link,thumbnail,sizeheight,sizewidth])
        #link 파일을 n.jpg 저장
        save_img(idx,link)
        #save_img(idx+1000,thumbnail)
    result = pd.DataFrame(image_list)
    result.to_csv('image/img_list.csv',index=False)
    print('▒▒▒▒▒이미지및 csv 파일 저장완료▒▒▒▒▒▒')
    return result

In [ ]:
get_image_list_save_img('청바지')

## 2.3 정규표현식을 활요한 데이터 수집
- RSS에서 search( 해당 패턴중 처음 내용의 일부), findall(해당 패턴 다 찾기)

In [ ]:
import re
parsing_str ="""010-9999-9999 02-888-1234
02-777-1212 <city>서울</city> <city>
용인</city>
<province>서울,인천,경기</province>
981212-1052009
"""

In [ ]:
phone = re.findall(r'\d{2,3}-\d{3,4}-\d{4}',parsing_str)
phone

In [ ]:
city = re.findall(r'<city>.+?</city>',parsing_str,re.DOTALL)
city

In [ ]:
phone = re.search(r'\d{2,3}-\d{3,4}-\d{4}',parsing_str) # 해당 정규표현식의 첫번째 아이만 찾아준다
phone

In [ ]:
phone.group(0)

In [ ]:
province= re.search(r'<province>(.+)</province>',parsing_str)
print('정규표현식 내용 전체 :',province.group(0))
print('정규표현식 내 괄호 안 추출 :',province.group(1))

In [ ]:
juminno = re.search(r'(\d{6})-(\d{7})',parsing_str)
print(juminno.group(0))
print(juminno.group(1))
print(juminno.group(2))


In [ ]:
# findall을 이용해서 특정문자 빼기=> replace('뺄문자','')
# 특수문자들 빼기 => findall
text = "이 영화는 좋아요★ 이 영화 추천합니다 ^ *$~ㅋㅋ"
temp = re.findall(r'[가-힣a-zA-Z0-9 ]',text)
print(''.join(temp))


### 1) 전국날씨 RSS를 정규표현식을 이용해서 가져오기

In [ ]:
import re
from urllib.request import urlopen
url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108'
target = urlopen(url)
contents = target.read().decode('utf-8')
#print(contents)
locations = re.findall(r'<location wl_ver="3">.+?</location>',contents,re.DOTALL)

for loc in locations:
    province = re.search(r'<province>(.+)</province>',loc)
    city = re.search(r'<city>(.+)</city>',loc)
    print(province.group(1),'지역의',city.group(1),'도시의 날씨입니다')
    data = re.findall(r'<data>.+?</data>',loc,re.DOTALL)
    for item in data:
        tmEf= re.search(r'<tmEf>(.+)</tmEf>',item).group(1)[:10] # 날짜
        time= int(re.search(r'<tmEf>(.+)</tmEf>',item).group(1)[11:13]) # 
        ampm = '오전' if time<12 else '오후'
        wf = re.search(r'<wf>(.+)</wf>',item).group(1) # 날씨
        tmn = re.search(r'<tmn>(.+)</tmn>',item).group(1) # 최저기온
        tmx = re.search(r'<tmx>(.+)</tmx>',item).group(1) # 최고기온
        rnSt = re.search(r'<rnSt>(.+)</rnSt>',item).group(1) # 강수확률
        print('\t날짜:{} {},날씨:{}, 기온:{}~{}, 강수확률:{}'.format(tmEf,ampm, wf,tmn,tmx,rnSt))

In [ ]:
# request.get() 방식으로 전송,
import re
import requests
from urllib.request import urlopen
items_list=[]
url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108'
# target = urlopen(url)
#contents = target.read().decode('utf-8')
target = requests.get(url)
#contents = target.content.decode('utf_8')
contents = target.text
locations = re.findall(r'<location wl_ver="3">.+?</location>',contents,re.DOTALL)

for loc in locations:
    province = re.search(r'<province>(.+)</province>',loc).group(1)
    city = re.search(r'<city>(.+)</city>',loc).group(1)
    data = re.findall(r'<data>.+?</data>',loc,re.DOTALL)
    for item in data:
        tmEf= re.search(r'<tmEf>(.+)</tmEf>',item).group(1)[:10] # 날짜
        time= int(re.search(r'<tmEf>(.+)</tmEf>',item).group(1)[11:13]) # 
        ampm = '오전' if time<12 else '오후'
        wf = re.search(r'<wf>(.+)</wf>',item).group(1) # 날씨
        tmn = re.search(r'<tmn>(.+)</tmn>',item).group(1) # 최저기온
        tmx = re.search(r'<tmx>(.+)</tmx>',item).group(1) # 최고기온
        rnSt = re.search(r'<rnSt>(.+)</rnSt>',item).group(1) # 강수확률
        items_list.append({
            '지역':province,
            '도시':city,
            '날짜':tmEf,
            'ampm':ampm,
            '날씨':wf,
            '최저기온':tmn,
            '최고기온':tmx,
            '강수확률':rnSt     
        })
import pandas as pd
df = pd.DataFrame(items_list)
df

In [ ]:
# 데이터 타입 변경 : astype (유효성 검사가 엄격)
    # astype(유효성검사가 엄격)
    # to_numeric, to_datetime (데이터가 불안정하거나 비표준형식일 때 편리)
df['날짜'].astype('datetime64[ns]')

In [ ]:
pd.to_datetime(df['날짜'])

In [ ]:
df['날짜']= pd.to_datetime(df['날짜'])
df.info()

In [ ]:
df['최저기온'] = df['최저기온'].astype('int')
df['최고기온'] = df['최고기온'].astype('int')
df['강수확률'] = pd.to_numeric(df['강수확률'])
df.info()

### 2)전국날씨 RSS를 BeautifulSoup을 이용한 xml크롤링

In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

items_list = []
url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108'
# target = urlopen(url)
# contents = target.read().decode('utf-8')
target = requests.get(url)
soup = BeautifulSoup(target.text, 'xml') # pip install lxml

locations = soup.select('location')
# print(locations[0])
for loc in locations:
    province = loc.select_one('province').text # 지역
    city = loc.select_one('city').text # 도시
    data = loc.find_all('data')
    for item in data:
        tmEf = item.select_one('tmEf').text # 날짜와 시간(여기서 오전오후 추출)
        ampm = '오전' if tmEf[11]=='0' else '오후'
        wf = item.select_one('wf').text # 날씨
        tmn = item.select_one('tmn').text # 최저기온
        tmx = item.select_one('tmx').text # 최고기온
        rnSt = item.select_one('rnSt').text # 강수확률
        items_list.append({
            '지역':province,
            '도시':city,
            '날짜':tmEf[:-6],
            'ampm':ampm,
            '날씨':wf,
            '최저':tmn,
            '최고':tmx,
            '강수확률':rnSt
        })
df = pd.DataFrame(items_list)
df['최저']=df['최저'].astype(np.int16)
df['최고']=df['최고'].astype(np.int16)
df['강수확률']=df['강수확률'].astype(np.int16)
df['날짜']=pd.to_datetime(df['날짜'])
df.head()

### 3) xml 응답하는 Open API 활용
- data.go.kr 에서 
    * 버스 위치 정보 조회 서비스 활용 신청(실시간 버스 운행 목록)
    * 서울특별시_노선정보조회 서비스 활용신청 (버스 id, 버스 정류장 목록)

In [ ]:
# step1. 버스 번호의 버스 id 받아오기
# 서울특별시_노선정보조회 서비스 - 4번() 기능 이용

In [106]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
from urllib.parse import quote
busNum = '162'
#busNum = '강남01'
# 인증키
key = 'kWiLqR5an3qgHfkycjPmjxAie5mGGzW0LuPktrSNA3kqpsmWfsLSW9TxR9k0khrZgJLecVcO88n5LsStz845eg%3D%3D'
# url1= f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?serviceKey={key}&strSrch={quote(busNum)}'
url1= f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?serviceKey={key}&strSrch={busNum}'
#response = requests.get(url1)
#soup = BeautifulSoup(response.text, "xml")
savefilename1 = 'data/ch14_1busInfo.xml'
urlretrieve(url1,savefilename1) # xml 파일을 내 컴에 savefilename1로 저장
xml = open(savefilename1,'r',encoding= 'utf-8').read()
soup = BeautifulSoup(xml,"xml")

for itemList in soup.select('itemList'):
    busRouteNm = itemList.find('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = itemList.find('busRouteId').text
        break
print('버스라우트id :',busRouteId)

버스라우트id : 100100034


In [ ]:
# step2. 해당 버스 id의 버스 정류장 목록 받아오기

In [107]:
import pandas as pd

url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
print(url2)
response = requests.get(url2)  # Ensure the variable is named 'response'
soup = BeautifulSoup(response.text, "xml")  # Correct use of .text
itemLists = soup.find_all('itemList')
bus_station=[]
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text
    station = itemList.find('station').text
    gpsX = itemList.find('gpsX').text
    gpsY = itemList.find('gpsY').text
    bus_station.append([stationNm,station,gpsX,gpsY])
df = pd.DataFrame(bus_station,columns=['정류장','id','경도','위도'])
df

http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey=kWiLqR5an3qgHfkycjPmjxAie5mGGzW0LuPktrSNA3kqpsmWfsLSW9TxR9k0khrZgJLecVcO88n5LsStz845eg%3D%3D&busRouteId=100100034


,정류장,id,경도,위도
0,정릉산장아파트,107000071,127.003343,37.616712
1,정릉4동주민센터.경국사,107000073,127.006345,37.613529
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899
3,성북청수도서관.정릉4동성당,107000075,127.0084193769,37.6115696748
4,정릉시장입구,107000077,127.0098212542,37.6084653256
...,...,...,...,...
72,성북청수도서관.정릉4동성당,107000076,127.009045,37.610876
73,북한산보국문역1번출구,107000519,127.008329146,37.6120835499
74,정릉4동주민센터.경국사,107000074,127.006681,37.613335
75,정릉대우아파트,107000072,127.00386,37.616708


In [ ]:
# step3. 해당 버스의 실시간 위치정보 받아오기 getBusPosByRtidList

In [108]:
key ='kWiLqR5an3qgHfkycjPmjxAie5mGGzW0LuPktrSNA3kqpsmWfsLSW9TxR9k0khrZgJLecVcO88n5LsStz845eg%3D%3D'
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey={key}&busRouteId={busRouteId}'
print(url3)
savefilename3 = 'data/ch14_3busPosition.xml'
urlretrieve(url3, savefilename3)
xml = open(savefilename3,'r',encoding='utf-8').read()
soup = BeautifulSoup(xml,"xml")
bus_position=[]

for itemList in soup.select('itemList'):
    plainNo= itemList.find('plainNo').text #차량번호
    congetion = itemList.find('congetion').text
    congetion = '여유' if congetion=='3' else '보통' if congetion =='4' else '혼잡' if congetion=='5' else '매우혼잡'
    gpsX = itemList.find('gpsX').text
    gpsY = itemList.find('gpsY').text
    lastStnId = itemList.find('lastStnId').text #최종 정류소 ID
    nextStId = itemList.find('nextStId').text #다음 정류소 ID
    nextStTm = itemList.find('nextStTm').text
    bus_position.append({
            '차량번호':plainNo,'혼잡도':congetion,'경도':gpsX,'위도':gpsY,'최종정류소ID':lastStnId,'다음정류소ID':nextStId,'도착소요시간':nextStTm
        
    })
df_pos= pd.DataFrame(bus_position)
print(f'{busNum}번 버스 운행중인 버스는{len(df_pos)}대 입니다' )
df_pos

http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey=kWiLqR5an3qgHfkycjPmjxAie5mGGzW0LuPktrSNA3kqpsmWfsLSW9TxR9k0khrZgJLecVcO88n5LsStz845eg%3D%3D&busRouteId=100100034
162번 버스 운행중인 버스는21대 입니다


,차량번호,혼잡도,경도,위도,최종정류소ID,다음정류소ID,도착소요시간
0,서울74사3381,여유,127.011763,37.604466,107000079,107000168,125
1,서울74사3030,여유,127.015301,37.596501,107000172,101000042,1721
2,서울74사2201,여유,127.001744,37.586243,100000005,101000042,1266
3,서울74사1967,여유,126.996958,37.577386,100000002,101000042,1035
4,서울70사6556,여유,126.98272,37.574611,100000107,101000042,498
5,서울74사3008,여유,126.982635,37.567936,101000141,101000042,207
6,서울74사1537,여유,126.972779,37.554163,101000004,102000018,395
7,서울74사1618,여유,126.968665,37.540095,102000045,118000061,522
8,서울74사3372,여유,126.963196,37.5364,102000050,118000061,319
9,서울70사6557,여유,126.92323,37.522601,118000076,118000070,167


In [126]:
df.loc[df['id']=='107000079', '정류장'].iloc[0]

'정릉우체국앞'

In [127]:
def station_name(row):
    row['최종정류소명'] = df.loc[df['id']==row['최종정류소ID'],'정류장'].iloc[0]
    row['다음정류소명'] = df.loc[df['id']==row['다음정류소ID'],'정류장'].iloc[0]
    return row

In [128]:
station_name(df_pos.loc[0])

차량번호        서울74사3381
혼잡도                여유
경도         127.011763
위도          37.604466
최종정류소ID     107000079
다음정류소ID     107000168
도착소요시간            125
최종정류소명         정릉우체국앞
다음정류소명       정릉입구.정릉역
Name: 0, dtype: object

In [129]:
df_pos = df_pos.apply(station_name,axis=1)
df_pos

,차량번호,혼잡도,경도,위도,최종정류소ID,다음정류소ID,도착소요시간,최종정류소명,다음정류소명
0,서울74사3381,여유,127.011763,37.604466,107000079,107000168,125,정릉우체국앞,정릉입구.정릉역
1,서울74사3030,여유,127.015301,37.596501,107000172,101000042,1721,돈암2동주민센터입구.흥천사,해운센터.롯데영플라자
2,서울74사2201,여유,127.001744,37.586243,100000005,101000042,1266,혜화동로터리.여운형활동터,해운센터.롯데영플라자
3,서울74사1967,여유,126.996958,37.577386,100000002,101000042,1035,창경궁.서울대학교병원,해운센터.롯데영플라자
4,서울70사6556,여유,126.98272,37.574611,100000107,101000042,498,안국동,해운센터.롯데영플라자
5,서울74사3008,여유,126.982635,37.567936,101000141,101000042,207,을지로입구역.광교,해운센터.롯데영플라자
6,서울74사1537,여유,126.972779,37.554163,101000004,102000018,395,서울역버스환승센터,남영역
7,서울74사1618,여유,126.968665,37.540095,102000045,118000061,522,용산경찰서,시범아파트.대교아파트
8,서울74사3372,여유,126.963196,37.5364,102000050,118000061,319,원효로2가,시범아파트.대교아파트
9,서울70사6557,여유,126.92323,37.522601,118000076,118000070,167,한국경제인협회,여의도역6번출구


# 연습문제
- YES24 베스트 셀러 정보 csv(txt)로 출력하기(https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=40)
- 순위, 책이름, 저자, 출판사, 가격(10위까지)


In [204]:
import requests
from bs4 import BeautifulSoup
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=40'
response = requests.get(url)
soup = BeautifulSoup(response.text,"html.parser")
book_list=[]
rank = soup.select('div.img_upper>em')
title = soup.select('div.info_row > a.gd_name' )
writer = soup.select('div.info_row > span.authPub.info_auth')
publisher = soup.select('div.info_row > span.authPub.info_pub')
price= soup.select('strong.txt_num> em.yes_b')
# 화면출력
ranks = [r.text for r in rank]
titles = [t.text for t in title]
writers = [w.text.split('\r\n')[0].strip() for w in writer]
publishers = [p.text for p in publisher]
prices = [p.text.strip() for p in price]

for rank,title,writer,publisher,price in zip(ranks,titles,writers,publishers,prices):
    book_list.append({'순위':rank,'제목':title,'저자':writer,'출판사':publisher,'가격':price})
df= pd.DataFrame(book_list)
df

,순위,제목,저자,출판사,가격
0,1,소년이 온다,한강 저,창비,"13,500"
1,2,채식주의자,한강 저,창비,"13,500"
2,3,작별하지 않는다,한강 저,문학동네,"15,120"
3,4,하루 한 장 나의 어휘력을 위한 필사 노트,유선경 저,위즈덤하우스,"21,420"
4,5,서랍에 저녁을 넣어 두었다,한강 저,문학과지성사,"10,800"
5,6,트렌드 코리아 2025,"김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 5명",미래의창,"18,000"
6,7,흰,한강 저/최진혁 사진,문학동네,"13,050"
7,8,산타 할아버지의 첫 크리스마스,맥 바넷 글/시드니 스미스 그림/김지은 역,책읽는곰,"13,500"
8,9,희랍어 시간,한강 저,문학동네,"11,700"
9,10,넥서스,유발 하라리 저/김명주 역,김영사,"25,020"


In [205]:
df.to_csv('data/ch14_예제.csv',index=False)